In [1]:
import psycopg2
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
#import constants
from datetime import date, datetime, timedelta
load_dotenv()
pd.options.mode.chained_assignment = None

from dateutil.relativedelta import relativedelta


In [2]:
from utils.db import Database

In [4]:
Database.initialize(
    database=os.getenv('WAREHOUSE_NAME'),
    user=os.getenv('WAREHOUSE_USER'),
    password=os.getenv('WAREHOUSE_PASSWORD'),
    host=os.getenv('WAREHOUSE_HOST'),
    port=os.getenv('WAREHOUSE_PORT'),
    minconn=1,
    maxconn=20
)

In [11]:
factory_name='大森 TIM BER'

In [12]:
selected_year = 2025

In [13]:
from utils.query.wh.extract import get_mtd_by_month, get_factory_code

In [14]:
factory_code = get_factory_code(factory_name)
# Get sales
df_sales = get_mtd_by_month(selected_year, 1, 31, factory_code, "fact_sales", "sales_quantity", "sales_date")
df_sales.columns = ['month','sales_quantity','sales_quantity_timber']
df_sales.drop(columns = ['sales_quantity'], inplace=True)


In [15]:
df_sales

,month,sales_quantity_timber
0,1,25590.0


In [16]:

# Get planned deliveries
df_plan  = get_mtd_by_month(selected_year, 1, 31, factory_code, "fact_order", "order_quantity", "estimated_delivery_date")
df_plan.columns = ['month','plan_quantity','plan_quantity_timber']
df_plan.drop(columns = ['plan_quantity'], inplace=True)


In [17]:
df_plan

,month,plan_quantity_timber
0,1,40697.0
1,2,33976.0
2,9,0.0


In [18]:
conn = Database.get_connection()
cur = conn.cursor()

In [29]:
cur.execute("""DELETE from fact_order where order_code = '2201-240829022-0006'""")

In [30]:
conn.commit()

In [27]:
data = cur.fetchall()
column_names = [description[0] for description in cur.description]
df = pd.DataFrame(data=data, columns=column_names)

In [28]:
df

,order_date,order_code,ct_date,factory_code,factory_order_code,tax_type,department,salesman,deposit_rate,payment_registration_code,...,package_order_quantity,delivered_package_order_quantity,unit,package_unit,estimated_delivery_date,original_estimated_delivery_date,pre_ct,finish_code,import_timestamp,import_wh_timestamp
0,2024-08-29,2201-240829022-0006,2024-08-29,31095,DHM-24/08-091,Thuế cộng riêng,業務部 NV,陳國勇,0,2004.0,...,1.0,1.0,kg,T,2025-09-04,2024-08-30,2101-220401022-0099,T? ??ng k?t thuc,2024-12-03 11:30:23.803593,2024-12-03 13:07:25.288145


In [25]:
df.to_excel('temp.xlsx',index=False)

In [ ]:

# Merge for plotting
df = df_sales.merge(df_plan, on='month', how='outer')
df.columns = ['month','sales_quantity','plan_quantity']

df.fillna(0,inplace=True)

df['percentage'] = df.apply(

    lambda row: 100 if row['plan_quantity'] == 0 else 
                0 if row['sales_quantity'] == 0 else 
                (row['sales_quantity'] / row['plan_quantity'] * 100),
    axis=1
)